In [1]:
from peewee import *
from models import *


db = SqliteDatabase("./space_tourism.db")
db.connect()

True

list(cursor.execute('''\
SELECT \
    t.name AS tour_name,\
    GROUP_CONCAT(DISTINCT t.date) AS tour_date,\
    AVG(tt.price) AS avg_ticket_price,\
    COUNT(tt.client_id) AS client_count\
FROM tours t\
JOIN tour_tickets tt ON t.id = tt.tour_id\
WHERE t.date > '2024-01-01'\
GROUP BY t.name\
HAVING COUNT(tt.client_id) > 2\
ORDER BY avg_ticket_price DESC;          \
                    \
'''))

In [2]:
query = (Tours
         .select(Tours.name.alias("tour_name"), fn.GROUP_CONCAT(Tours.date.distinct()).alias("tour_data"),
                 fn.AVG(TourTickets.price).alias("avg_ticket_price"),
                 fn.COUNT(TourTickets.client_id).alias("client_count"))
         .join(TourTickets)
         .where(Tours.date > "2024-01-01")
         .group_by(Tours.name)
         .having(fn.COUNT(TourTickets.client) > 2)
         .order_by(SQL('tour_name').desc())

         )

for i in query:
    print(i.tour_name, i.tour_data, i.avg_ticket_price, i.client_count)

Mars Expedition 2025-07-26,2025-05-05 14519.333333333334 3
Jupiter Adventure 2025-05-20,2025-06-25 13786.555555555555 9
ISS Experience 2025-07-31,2025-07-06,2025-07-20,2025-06-12 13182.125 16


list(cursor.execute('''\
SELECT \
    c.name || ' ' || c.surname AS client_name,\
    r.text AS review_text,\
    r.date AS review_date,\
    t.name AS tour_name,\
    AVG(tt.price) AS avg_tour_price\
FROM clients c\
JOIN reviews r ON c.id = r.client_id\
JOIN tours t ON r.tour_id = t.id\
JOIN tour_tickets tt ON t.id = tt.tour_id\
GROUP BY c.id\
HAVING avg_tour_price > (SELECT AVG(price) FROM tour_tickets)\
ORDER BY avg_tour_price DESC\
LIMIT 10;
               
                    
'''))

In [51]:
query = (Clients
         .select(Clients.name, Clients.surname,
                 Reviews.text.alias("review_text"),
                 Reviews.date.alias("review_date"),
                 Tours.name.alias("tour_name"),
                 fn.AVG(TourTickets.price).alias("avg_tour_price"))
         .join(Reviews)
         .join(Tours)
         .join(TourTickets)
         .group_by(Clients.id)
         .having(SQL("avg_tour_price") > (TourTickets.select(fn.AVG(TourTickets.price))))
         .order_by(SQL("avg_tour_price").desc())
         .limit(10)
        )

for i in query.namedtuples():
    print(i.name + " " + i.surname, f"'{i.review_text}'", i.review_date, i.tour_name, i.avg_tour_price)

John Williams 'The food could be better.' 2025-04-21 ISS Experience 16479.0
William Smith 'The crew was fantastic.' 2025-04-20 Jupiter Adventure 16004.777777777777
Michael Brown 'Worth every penny!' 2025-03-25 Jupiter Adventure 15412.0
John Jones 'Amazing experience!' 2025-04-12 Jupiter Adventure 15412.0
Sophia Smith 'Worth every penny!' 2025-04-08 Jupiter Adventure 15412.0
Sophia Jones 'The crew was fantastic.' 2025-03-24 Jupiter Adventure 13786.555555555555


list(cursor.execute('''\
SELECT \
    c.name || ' ' || c.surname AS Candidats_name,\
    p.name AS position_name,\
    c.mark AS Candidats_mark,\
    AVG(c2.mark) AS avg_position_mark,\
    COUNT(e.id) AS current_employees_in_position\
FROM candidats c\
JOIN positions p ON c.position_id = p.id\
LEFT JOIN employees e ON e.position_id = p.id\
JOIN candidats c2 ON c2.position_id = p.id\
GROUP BY c.id\
HAVING c.mark > (SELECT AVG(mark) FROM candidats WHERE position_id = p.id)\
ORDER BY c.mark DESC;\
               
                    
'''))

In [66]:
# Создаем алиас для таблицы кандидатов (для c2 в оригинальном запросе)
Candidats2 = Candidats.alias('c2')

subquery = (Candidats
           .select(fn.AVG(Candidats.mark))
           .where(Candidats.position_id == Positions.id))

query = (Candidats
    .select(
        (Candidats.name + ' ' + Candidats.surname).alias('Candidats_name'),
        Positions.name.alias('position_name'),
        Candidats.mark.alias('Candidats_mark'),
        fn.AVG(Candidats2.mark).alias('avg_position_mark'),
        fn.Count(Employees.id).alias('current_employees_in_position')
    )
    .join(Positions, on=(Candidats.position == Positions.id))
    .switch(Candidats)  # Возвращаемся к основной таблице для следующего соединения
    .join(Employees, join_type='LEFT', on=(Employees.position == Positions.id))
    .switch(Candidats)  # Возвращаемся к основной таблице
    .join(Candidats2, on=(Candidats2.position == Positions.id))
    .group_by(Candidats.id)
    .having(Candidats.mark > subquery)
    .order_by(Candidats.mark.desc())
)

# Для выполнения запроса
results = list(query.dicts())

OperationalError: near "AS": syntax error

list(cursor.execute('''\
SELECT \
    s.name AS spaceship_name,\
    SUM(p.price * r.count) AS total_maintenance_cost\
FROM spaceships s\
JOIN maintain_tickets mt ON s.id = mt.spaceship_id\
JOIN reasons r ON mt.reason_id = r.id\
JOIN products p ON r.product_id = p.id\
GROUP BY s.id\
ORDER BY total_maintenance_cost DESC;\
               
                    
'''))

list(cursor.execute('''\
SELECT \
    a.name AS airport_name,\
    COUNT(t.id) AS tour_count,\
    GROUP_CONCAT(DISTINCT so.name) AS space_objects,\
    SUM(s.human_places) AS total_passenger_capacity\
FROM airports a\
JOIN tours t ON a.id = t.airport_id\
JOIN space_objects so ON t.obj_id = so.id\
JOIN spaceships s ON t.spaceship_id = s.id\
GROUP BY a.id\
ORDER BY tour_count DESC;\
                    
'''))